In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import timm
from EFF3D.MRI import *
from fastai.vision.all import *
from EFF3D.vit import MRIVisionTransformer
import torch.nn as nn
from sam import SAM

In [ ]:
torch.cuda.set_device(1)

In [ ]:
df=pd.read_csv('../nonorm/ADNCold.csv')
# path=Path('/home/staff/xin/Downloads/newMRI/ADtrain')

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MriTensorImage.create)),CategoryBlock),
               get_x=ColReader('name'),
               get_y=ColReader('label'),
               splitter=RandomSplitter(valid_pct=0.2,seed=2),
              )

dls=db.dataloaders(source=df, bs=12, num_workers=4)

In [ ]:
sort=np.load('./sort.npy')
model=MRIVisionTransformer(mask=sort,num_heads=4,depth=6,num_patches=100)

In [ ]:
class TstCallback(Callback):
    def before_fit(self):
        nowdlist=['pos_embed', 'cls_token', 'dist_token']
        for name,p in self.named_parameters():
            if name in nowdlist:
                self.opt.state[p]['do_wd'] = False

In [ ]:
class SAM(Callback):
    "Sharpness-Aware Minimization"
    def __init__(self, zero_grad=True, rho=0.05, eps=1e-12, **kwargs): 
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"
        self.state = defaultdict(dict)
        store_attr()

    def params(self): return self.learn.opt.all_params(with_grad=True)
    def _grad_norm(self): return torch.norm(torch.stack([p.grad.norm(p=2) for p,*_ in self.params()]), p=2)
    
    @torch.no_grad()
    def first_step(self):
        print('here')
        scale = self.rho / (self._grad_norm() + self.eps)
        for p,*_ in self.params():
            self.state[p]["e_w"] = e_w = p.grad * scale
            p.add_(e_w)  # climb to the local maximum "w + e(w)"
        if self.zero_grad: self.learn.opt.zero_grad()
        
    @torch.no_grad()    
    def second_step(self):
        for p,*_ in self.params(): p.sub_(self.state[p]["e_w"])

    def before_step(self, **kwargs):
        self.first_step()
        self.learn.pred = self.model(*self.xb); self.learn('after_pred')
        self.loss_func(self.learn.pred, *self.yb).backward()
        self.second_step()

In [ ]:
learn=Learner(dls,model=model,loss_func=LabelSmoothingCrossEntropy(),metrics=accuracy,cbs=SAM)

In [ ]:
learn.fit_flat_cos(1,1e-4)

epoch,train_loss,valid_loss,accuracy,time


here
here
here
here


KeyboardInterrupt: 